In [ ]:
###### import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import unicodedata
import re
import time 
from datetime import datetime, timedelta
from threading import Timer

#Schedule run every 7 days at 1am 

x=datetime.today()
y = x.replace(day=x.day, hour=1, minute=0, second=0, microsecond=0) + timedelta(days=7)
delta_t=y-x
secs=delta_t.total_seconds()

def research():
    stflinks = []
    stfname = []
    name = []
    coven = pd.DataFrame(columns= ['name', 'link','research_interest'])
    page1 = requests.get("https://scholar.google.co.uk/citations?view_op=view_org&hl=en&org=9117984065169182779")
    soup1 = BeautifulSoup(page1.text, 'html.parser')
    #sub link to a staff page

    for main in soup1.find_all('div', class_='gsc_1usr'):
        try:
            slink = main.h3.a['href']
        except Exception as e:
            slink = 'None' 
        stflinks.append(slink)

    #staff page extract details(name and research area)   
    for stfpage in stflinks:
        pagestf= requests.get("https://scholar.google.co.uk" + str(stfpage))
        soup2 = BeautifulSoup(pagestf.text, 'html.parser')                      
        try:
            sname = soup2.find('div', id ='gsc_prf_in').text
        except Exception as e:
            sname = 'None'  

    #all area research in one row         
        stfarea = []
        for main1 in soup2.find_all(class_='gsc_prf_inta gs_ibl'):
            try:
                stfar = main1.text
            except Exception as e:
                stfar = 'None'
            #stfarea.append('\n')    
            stfarea.append(stfar)


        links = ("https://scholar.google.co.uk" + str(stfpage))
        string = ", "
        string = string.join(stfarea)
    #Dataframe     
        a = {'name': sname, 'link' : links, 'research_interest': string}
        cov = pd.DataFrame.from_dict(a, orient='index')
        coventry = cov.transpose()    
        coven = coven.append(coventry, ignore_index= 'True')
        coven.to_csv('coventry1.csv')


    for main3 in soup1.find_all('div', class_= 'gsc_pgn'):
        for main3 in soup1.find_all(class_= 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'):
            try:
                nextlink = main3['onclick']            
            except Exception as e:
                nextlink = 'None' 
            start_pos = nextlink.find('=')
            temp= nextlink[start_pos + 2:len(nextlink) -1]

    def normalize_string(text):
        
        text = text.replace("\\x3d", "=")    
        text = text.replace("\\x26", "&")    
        text = text.strip()

        text = text.rstrip('.')
        text = unicodedata.normalize(u'NFKD', str(text)).encode('ascii', 'ignore').decode('utf8')

        return text 
    normalize_string(temp)

    link = 'https://scholar.google.co.uk' + normalize_string(temp)
    #print(link)
    
    
#Using the link to to the next page
    i= 1
    while i: 
        if nextlink != 'None':
            stflinks = []
            stfname = []
            name = []
            coven = pd.DataFrame(columns= ['name', 'link','research_interest'])
            page1 = requests.get(link)
            soup1 = BeautifulSoup(page1.text, 'html.parser')
            #sub link to a staff page

            for main in soup1.find_all('div', class_='gsc_1usr'):
                try:
                    slink = main.h3.a['href']
                except Exception as e:
                    slink = 'None' 
                stflinks.append(slink)

            #staff page extract details(name and research area)   
            for stfpage in stflinks:
                pagestf= requests.get("https://scholar.google.co.uk" + str(stfpage))
                soup2 = BeautifulSoup(pagestf.text, 'html.parser')                      
                try:
                    sname = soup2.find('div', id ='gsc_prf_in').text
                except Exception as e:
                    sname = 'None'  

            #all area research in one row         
                stfarea = []
                for main1 in soup2.find_all(class_='gsc_prf_inta gs_ibl'):
                    try:
                        stfar = main1.text
                    except Exception as e:
                        stfar = 'None'
                    #stfarea.append('\n')
                    stfarea.append(stfar)


                links = ("https://scholar.google.co.uk" + str(stfpage))
                string = ", "
                string = string.join(stfarea)
            #Dataframe     
                a = {'name': sname, 'link' : links, 'research_interest': string}
                cov = pd.DataFrame.from_dict(a, orient='index')
                coventry = cov.transpose()    
                coven = coven.append(coventry, ignore_index= 'True')
            coven.to_csv('coventry1.csv', mode='a', header=False)

            #Nextpage link
            for main3 in soup1.find_all('div', class_= 'gsc_pgn'):
                for main3 in soup1.find_all(class_= 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'):
                    try:
                        nextlink = main3['onclick']            
                    except Exception as e:
                        nextlink = 'None' 
                    start_pos = nextlink.find('=')
                    temp= nextlink[start_pos + 2:len(nextlink) -1]

            def normalize_string(text):

                text = text.replace("\\x3d", "=")    
                text = text.replace("\\x26", "&")    
                text = text.strip()

                text = text.rstrip('.')
                text = unicodedata.normalize(u'NFKD', str(text)).encode('ascii', 'ignore').decode('utf8')

                return text 
            normalize_string(temp)

            link = 'https://scholar.google.co.uk' + normalize_string(temp)
            print(link)
        else:
            break
        #random pause between 1 to 5 sec 
        sleep(randint(1,5))        
        i += 1   
        
        
t = Timer(secs, research)
t.start()

In [ ]:
##Schedule crawling on every 7 days at 1am 
x=datetime.today()
y = x.replace(day=x.day, hour=1, minute=0, second=0, microsecond=0) + timedelta(days=7)
delta_t=y-x
secs=delta_t.total_seconds()

def research():
    #the seed url will output a link to be used to go to the next page
    i= 1
    while i: 
        if nextlink != 'None':
            stflinks = []
            stfname = []
            name = []
            coven = pd.DataFrame(columns= ['name', 'link','research_interest'])
            page1 = requests.get(link)
            soup1 = BeautifulSoup(page1.text, 'html.parser')                   
        else:
            break
        #random crawler pause between 1 to 5 sec after each page 
        sleep(randint(1,5))        
        i += 1       
        
t = Timer(secs, research)
t.start()